In [1]:
import pandas as pd
import numpy as np 
import re, json, os , logging , random, subprocess
from lib.config import connstr
from lib.context import context
from lib.utils import * 
from lib.picklist_recommender import picklist_recommender
from lib.issue_provider import issue_provider
from lib.questionnaire_parser import questionnaire_parser
from lib.questionnaire_picklist_parser import questionnaire_picklist_parser
from lib.script_generator import script_generator   
import nltk
from nltk.corpus import stopwords  
from nltk.stem import PorterStemmer
ps = PorterStemmer()
sw=stopwords.words('english') 
ctx=context() 
ctx.logger.setLevel(logging.DEBUG)
config = {}
with open('config.json', 'r') as f: 
    config=json.loads(f.read())    
ctx.config=config 
import warnings
warnings.filterwarnings('ignore')
#df, code = generate_code_from_db(ctx, qgroup=4028)

In [ ]:
s =''
f=3411
t=14920
for i in range(1,11):
    
    s=s+f"SET @RESULT = (SELECT REPLACE(@RESULT, '3414', '14921'))"
print(s)

In [2]:
def to_proper(s):
    n=''
    for i in range(len(s[:-1])):  
        n=n+s[i]
        if ord(s[i]) > 96 and ord(s[i+1]) < 97: n=n+ ' '
    return n+s[-1]    
        
def normalizedf(df):
    df.drop( columns=[c for c in df.columns if '__' in c ] , inplace=True )   
    cols = list(df)
    if 'PK_PICKLISTTYPE' not in cols: df['PK_PICKLISTTYPE'] = 0
    if 'HEADER' not in cols: df['HEADER'] = df['COLUMN_NAME'].apply(to_proper)
    if 'DATA_TYPE' not in cols: df['DATA_TYPE'] = 'NVARCHAR'
    if 'CHARACTER_MAXIMUM_LENGTH' not in cols: df['CHARACTER_MAXIMUM_LENGTH'] = 4000
    df['DATA_TYPE']=df['DATA_TYPE'].str.upper()  
    df['CHARACTER_MAXIMUM_LENGTH']=df['CHARACTER_MAXIMUM_LENGTH'].fillna(0).astype(int)
    df['PK_PICKLISTTYPE']=df['PK_PICKLISTTYPE'].fillna(0).astype(int)
    df['CHARACTER_MAXIMUM_LENGTH']=df['CHARACTER_MAXIMUM_LENGTH'].apply(lambda s: 4000 if s < 0 else s )
    df[cols] = df[cols].astype(str) 
    df=df.loc[:, ['COLUMN_NAME','DATA_TYPE','CHARACTER_MAXIMUM_LENGTH','HEADER', 'PK_PICKLISTTYPE' ]]
    return df

In [ ]:
def df_fromDataFields(path): 
    lst=[]
    cols=[]
    with open(path, 'r') as f:  
        g=re.search("<DataFields>(.*)<\/Data", f.read() , flags=re.DOTALL)   
        lst=g.groups(0)[0].split('<CB:DataField')[1:] 
        for l in lst:
            cols.append({
                'COLUMN_NAME':re.search('DBColumnName="(\w+)"\s', l ).groups(0)[0]
                ,'HEADER':re.search('ImportColumnName="([\w|\s]+)"', l ).groups(0)[0]
                })
            
    return normalizedf(pd.DataFrame(cols))
df = df_fromDataFields(r'D:\dev\CyberScope\CyberScopeBranch\CSwebdev\code\CyberScope\FismaForms\2024\2024_Q3_CIO_1C.aspx')

df
# print( SQL_INSERT_FROM_DF(df, 'FEDRAMP_UPLOAD')[1] ) 

In [ ]:
df = pd.read_excel(r'C:\Users\Tim\Downloads\fedramp.xlsx', sheet_name='Agency - CSP Table', header=1, usecols='B:Z' )  
df.columns=[c.replace(' ' ,'') for c in df.columns] 
df
# df.to_excel(r'C:\Users\Tim\Downloads\fedramp_admin_import.xlsx', sheet_name='Admin Import', index=False)


In [ ]:
df = pd.read_excel(r'C:\Users\Tim\Downloads\FedRAMP Cloud Services - 060524.xlsx', sheet_name='FedRAMP CSP Masterlist', header=0, usecols='A:Z' ) 
for c in df.columns:
    s = f'<CB:DataField  DBColumnName="{c.replace(" ","")}" Require="False" ImportColumnName="{c}" runat="server"/>  '
    if "Date" in c:
        s = f'<CB:DataField  DBColumnName="{c.replace(" ","")}" Require="False" DataType="System.DateTime" NonDatePlaceholder="01/01/1900" ImportColumnName="{c}"  runat="server"/>  '
    print(s)

In [ ]:
fields = pd.read_csv(f'{ctx.get_dest()}fedramp.csv') 
df=normalizedf(fields)
df

In [ ]:
ag =  set(sql_todf("  SELECT Component FROM [Component List] WHERE IsActive=1 AND FK_PK_Component IS NULL  ", connstr)['Component']) 
ag = set(ag)
ag

In [ ]:
df=sql_todf("  SELECT * FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'TIC_CloudServices'  ", connstr)   
df=normalizedf(df) 
df 

In [ ]:
for c in df.columns:
    s = f'<CB:DataField  DBColumnName="{c.replace(" ","")}" Require="False" ImportColumnName="{c}" runat="server"/>  '
    if "Date" in c:
        s = f'<CB:DataField  DBColumnName="{c.replace(" ","")}" Require="False" ImportColumnName="{c}"  runat="server"/>  '
    print(s)

In [ ]:
for i, r in df.iterrows(): 
    DATA_TYPE=r["DATA_TYPE"]
    COLUMN_NAME=r["COLUMN_NAME"]
    HEADER=r["HEADER"]
    PK_PICKLISTTYPE=str(r["PK_PICKLISTTYPE"] )
    s=f'<CB:DataField DBColumnName="{COLUMN_NAME}" ImportColumnName="{HEADER}" SprocParamName="{COLUMN_NAME}" runat="server"/>' 
    if PK_PICKLISTTYPE != '0': s=s.replace('runat',' PickListTypeID="'+ PK_PICKLISTTYPE +'" runat')   
    
    e=f',{COLUMN_NAME} AS [{HEADER}] '
    if PK_PICKLISTTYPE != '0': e=f', (SELECT DisplayValue FROM vwPICKLISTS WHERE PK_Picklist={COLUMN_NAME}) AS [{HEADER}]' 
    if 'Date' in COLUMN_NAME:
        e=f" , CASE WHEN {COLUMN_NAME} LIKE '%2000%' THEN 'NA' ELSE FORMAT({COLUMN_NAME}, 'MM/dd/yyyy') END AS [{HEADER}]"
    print(s)

In [ ]:

for i, row in df[20:].iterrows():
    with open(ctx.get_tempalte_dir()+'GridTemplateColumn.aspx', 'r') as f: 
        s=f.read() 
    DATA_TYPE=row['DATA_TYPE']
    COLUMN_NAME=row['COLUMN_NAME']
    HEADER=row['HEADER']
    PK_PICKLISTTYPE=row['PK_PICKLISTTYPE']
    s=s.replace('{COLUMN_NAME}', COLUMN_NAME)
    if 'INT' in DATA_TYPE:  
        s=s.replace('{Bind_COLUMN_NAME}', COLUMN_NAME+'_Display')
        s=s.replace('<%-- --%>', f'<telerik:RadDropDownList ID="{COLUMN_NAME}" DataValueField="YN" runat="server" />')  
    else:
        s=s.replace('{Bind_COLUMN_NAME}', COLUMN_NAME )
        if 'Date' in COLUMN_NAME:  
            # print("const "+COLUMN_NAME+" = new CBDatePicker({ id: $(`[id$=_"+COLUMN_NAME+"]`).prop('id'), allowNa: true  }); ")  
            s=s.replace('<%-- --%>', f'<asp:HiddenField ID="{COLUMN_NAME}" runat="server" />') 
        else: 
            s=s.replace('<%-- --%>', f'<telerik:RadTextBox ID="{COLUMN_NAME}" runat="server" />') 
    #print(f"[{COLUMN_NAME}] [NVARCHAR](4000) NULL,")  
    #print('{ field: "'+COLUMN_NAME+'", title: "'+HEADER+'" },')
    print (s)

In [ ]:
 
def get_gridcols(df,out=False):
    ldf=df.to_dict(orient='records') 
    code,ddl,dv,sel,dtf=[],[],[],[],[]
    with open(ctx.get_tempalte_dir()+'GridTemplateColumn.aspx', 'r') as f: 
        t=f.read() 
        
        for d in ldf: 
            col=d['COLUMN_NAME'] 
            s=t.replace("{COLUMN_NAME}",d['COLUMN_NAME'])  
            if 'INT' in d['DATA_TYPE'].upper():
                s=s.replace('{Bind_COLUMN_NAME}',f"{d['COLUMN_NAME']}_Display")
                s=s.replace('<%-- --%>', f'<telerik:RadDropDownList ID="{col}" DataValueField="YN" runat="server" />')
                ddl.append(f'BindDDL(e, "{col}", "{col}", "{col}")')
                sel.append(f',(SELECT DisplayValue FROM vwPICKLISTS WHERE PK_Picklist={col}) AS {col}') 
            else: 
                s=s.replace('{Bind_COLUMN_NAME}',f"{d['COLUMN_NAME']}")
                s=s.replace('<%-- --%>', f'<telerik:RadTextBox ID="{col}" runat="server" />')
                dv.append(f'DirectCast(e.Item.FindControl($"{col}"), RadTextBox).Text = drVal("{col}").ToString()')
                sel.append(f',{col}') 
            code.append(f'{s}')     
    code='\n'.join(code) 
    ddl='\n'.join(ddl)
    dv= '\n'.join(dv)
    sel= '\n'.join(sel)[1:] 
    return {'code':code, 'ddl':ddl,'dv':dv,'sel':sel, 'dtf': dtf}
  
 
p = get_gridcols(df, out=False)
p
print ( p['code'] ) 
 

In [ ]:
sql = get_gridcols(df)
print( sql['code'])

In [ ]:
def get_sql(df,out=False):
    parms=[]
    for i,r in df.iterrows():
        s=f"@{r['COLUMN_NAME']} {r['DATA_TYPE'].upper()}({r['CHARACTER_MAXIMUM_LENGTH']}) = NULL"
        if r['DATA_TYPE'].upper() == 'INT':
            s=f"@{r['COLUMN_NAME']} {r['DATA_TYPE'].upper()} = NULL"  
        parms.append(f',{s}')

    into=', '.join([c for c in df['COLUMN_NAME'] if 'PK_' not in c])
    vals=', @'.join([c for c in df['COLUMN_NAME'] if 'PK_' not in c]) 
    ins=f'\n INSERT INTO @TABLE ({into}) \n VALUES (@{vals})'

    parms='\n'.join(parms)[:] 
    update=', '.join([f'{c}=@{c}' for c in df['COLUMN_NAME'] if 'PK_' not in c ])  
    ext=' AND ' + ' AND '.join([f'{c}=@{c}' for c in df['COLUMN_NAME'] if 'PK_' not in c ])

    sel=[] 
    exp=[]
    for i,r in df.iterrows():   
        s=f"\n, {r['COLUMN_NAME']} "
        if r['DATA_TYPE'].upper() == 'INT':   
            s=s+f"\n, (SELECT DisplayValue FROM vwPicklists WHERE PK_Picklist={r['COLUMN_NAME']}) AS [{r['COLUMN_NAME']}_Display]" 
            exp.append(f"\n, (SELECT DisplayValue FROM vwPicklists WHERE PK_Picklist={r['COLUMN_NAME']}) AS [{r['HEADER']}]")
        elif r['DATA_TYPE'].upper() == 'DATETIME':
            s=s+f"\n, FORMAT({r['COLUMN_NAME']}, 'MM/dd/yyyy') AS [{r['COLUMN_NAME']}_Display] " 
            exp.append(f"\n, FORMAT({r['COLUMN_NAME']}, 'MM/dd/yyyy') AS [{r['HEADER']}]") 
        else:
            exp.append(f"\n, {r['COLUMN_NAME']} AS [{r['HEADER']}]")
        sel.append(''+s)
    sel=''.join(sel)  
    exp=''.join(exp)  
    return {'parms':parms,'ins':ins, 'update':update, 'sel':sel, 'ext':ext, 'exp':exp  }


# df = sql_todf("SELECT * FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME='FEDRAMP_UPLOAD'", ctx.connstr) 
# df = normalizedf(df) 
#  df.columns
d=get_sql(df, out=False)
print( d['ins']  )

In [ ]:
df=sql_todf("  SELECT * FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'FEDRAMP_UPLOAD'  ", connstr)   
for i, r in df.iterrows(): 
    DATA_TYPE=r["DATA_TYPE"]
    COLUMN_NAME=r["COLUMN_NAME"]
    print( f',{COLUMN_NAME}' )

In [ ]:
df = sql_todf("SELECT * FROM TIC_CloudServices WHERE COMMENTS LIKE '##%'", ctx.connstr) 
df

In [ ]:
df = pd.read_excel(r'C:\Users\Tim\Downloads\OMB-FedRAMP-CSP-JSON-CSV Analysis - FedRAMP 051724 (1).xlsx', sheet_name='Agency Table', header=0, usecols='A:Z' ) 
df
 

In [ ]:
for c in df.columns:
    s = f'<CB:DataField  DBColumnName="{c.replace(" ","")}" Require="False" ImportColumnName="{c}" runat="server"/>  '
    if "Date" in c:
        s = f'<CB:DataField  DBColumnName="{c.replace(" ","")}" Require="False" ImportColumnName="{c}"  runat="server"/>  '
    print(s)